In [1]:
import re
import random
import time
import datetime
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True,formatter={'float_kind': '{:0.2f}'.format})

import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common import by
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'webdriver_manager'

In [2]:
from fake_useragent import UserAgent
from stem import Signal
from stem.control import Controller

In [3]:
# download tor homebrew install tor
# start tor in terminal by 'tor'
# congfigure /opt/homebrew/etc/tor/torrc file
# enable ControlPort 9051
# enable CookieAuthentication 1
# change / set tor password : in terminal tor --hash-password mypassword
def renew_proxy():
    with Controller.from_port(port = 9051) as c:
        c.authenticate(password='nhhtq103')
        c.signal(Signal.NEWNYM)# change ip address
    proxy = {
        'http':  'socks5://localhost:9050',
        'https': 'socks5://localhost:9050',
    }
    r = requests.get('https://httpbin.org/get', proxies=proxy)  # using TOR network
    print( r.json()['origin'])
    return proxy 

In [33]:
shop_name=random.choice(['relaxdays','buddentown','lumo-shop','jamega-gmbh','mumbi-shop','kw-commerce','hausundgarten'])
shop_name

'lumo-shop'

In [5]:
def split_number(text):
    
    match = re.search(r'[0-9]{12}',str(text))
    return match.group() if match else None

In [6]:
def look_up_item_with_google(item):
    '''this function imitate user click from google to ebay'''
    
    item_url=f'https://www.ebay.de/itm/{item}'
    google='https://www.google.de'
    driver = webdriver.Chrome("../../chromedriver")
    driver.get(google)
    pause=np.random.choice(np.linspace(0.0,3.0,50))
    try:
        driver.find_element(by=By.XPATH,value='//*[@id="L2AGLb"]/div').click()
        element = driver.find_element(by=By.XPATH,value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
        element.send_keys(item_url)
        element.send_keys(Keys.RETURN)
        driver.find_element(by=By.XPATH,value='//a[@href="'+item_url+'"]').click()
    except NoSuchElementException:
        driver.get(item_url)
    time.sleep(pause)
    driver.execute_script("window.scrollBy(0,2100)","")
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    sales_each_day = retrieve_sale_each_day(soup)
    time.sleep(pause)
    print(sales_each_day)
    driver.find_element(by=By.XPATH,value='//*[@id="RightSummaryPanel"]/div[4]/div[1]/div/div[3]/div/div[2]/a/span[1]/span').click()



In [30]:
def retrieve_item_data_by_shopname(shopname,test=True,no=5):
    
    
    price_list,url_list,shipping_list,item_number_list,item_titel_list=[],[],[],[],[]
    lst=(item_number_list,price_list,shipping_list,item_titel_list,url_list)
    max_item = no if test else 50
    for i in range(1,2):#max_item):
        time.sleep(np.random.choice(np.linspace(0.0,3.0,50)))
        url=f'https://www.ebay.de/sch/i.html?_dmd=2&iconV2Request=true&_ssn={shopname}&store_cat=0&store_name={shopname}&_oac=1&_ipg=240&_pgn='
        # pages
        url = url+str(i)
        proxy=renew_proxy()
        result = requests.get(url,proxies=proxy)
        src = result.content
        soup = BeautifulSoup(src,'html.parser')
        print(result.content)
        li = soup.find_all('li',class_='s-item')

        #first article is logo
        for i in range(1,len(li)): 
            item_title=li[i].findChildren('h3')[0].text
            item_link=li[i].find('a',class_='s-item__link')['href'].split('?')[0]
            item_number=split_number(item_link)

            item_price=li[i].findChildren('span',class_="s-item__price")[0].text
            try:
                item_shipping=li[i].findChildren('span',class_='s-item__shipping')[0].text
            except IndexError:
                item_shipping=0    

            item_data=[item_number,item_price,item_shipping,item_title,item_link]
            for ida in item_data:
                ida = ida if ida else np.nan
            for item,item_list in zip(item_data,lst):
                item_list.append(item)
    dic={'price':price_list,'shipping':shipping_list,'title':item_titel_list,'item_number':item_number_list,'url':url_list}
    return pd.DataFrame(dic)


In [8]:
def fetch_item_number(shop_url):
    
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src,'lxml')
    items=[]
    for a in soup.find_all('a', href=True):
        item=split_number(a)
        if item:items.append(item)
    return np.unique(items)

In [9]:
def retrieve_shop_itemnnumber(shopname,max_item=240):
    ''' max_item can be all or a number'''
    url=f'https://www.ebay.de/sch/i.html?_dmd=2&iconV2Request=true&_ssn={shopname}&store_cat=0&store_name={shopname}&_oac=1&_ipg=240&_pgn='
    item_list=[]
    if max_item=='all':
        for i in range(1,100):
            items = fetch_item_number(url+str(i))
            item_list.extend(items)
    else:
        limit= np.ceil(max_item/240)+1
        for i in range(1,int(limit)):
            items = fetch_item_number(url+str(i))
            item_list.extend(items)
    return item_list

In [10]:
def find_item_number_column(data):
    '''define column with item number in it '''
    for column in data.columns:
        item_exist=split_number(data.loc[0,column])
        if item_exist:
            return column
        else :
            raise ValueError('column not exist')
            

In [9]:

def retrieve_total_sales(soup_instant):
    #retrieve Gesamtverkauf
    total_solds_tag =soup_instant.select('form > div > div > div > div > div > span > span > span > a')    
    if not total_solds_tag:
        total_solds_tag =soup_instant.select('form > div > div > div > div > span > span > span > a') 

    total_sold = total_solds_tag[0].getText() if total_solds_tag else np.nan       
    return total_sold

def retrieve_sale_each_day(soup_instant):
    #retrieve Verkäufe pro Tag
    solds_tag = soup_instant.select("#vi_notification_new > span")
    sold = solds_tag[0].getText() if solds_tag else 0 
    return sold 

def retrieve_price(soup_instant):
    #retrieve Preis
    prices_tag = soup_instant.select("#prcIsum")
    price = prices_tag[0].getText() if prices_tag else np.nan        
    return price

def retrieve_inventory(soup_instant):
    #retrieve Verfügbarkeit
    inventory_tag=soup_instant.select("#qtySubTxt > span")
    inventory = inventory_tag[0].getText() if inventory_tag else np.nan       
    return inventory

def retrieve_shipping(soup_instant):
    #retrieve shipping cost
    shippings_tag=soup_instant.select("#fshippingCost > span")
    shipping = shippings_tag[0].getText() if shippings_tag else 0       
    return shipping

def retrieve_title(soup_instant):
    #retrive title of the item
    title_tag = soup_instant.select("#LeftSummaryPanel > div.vi-swc-lsp > div:nth-child(1) > div > h1 > span")
    title =title_tag[0].getText() if title_tag else np.nan
    return title

def retrieve_company_name(soup_instant):
    company_name_tag = soup_instant.select("#vi-desc-maincntr > div.bsi-noexp.bsi-noexp-toc > div > div > div.ux-section.ux-section--nameAndAddress > div > div:nth-child(1) > span")
    company_name =company_name_tag[0].getText() if company_name_tag else np.nan
    return company_name

def retrieve_img(soup_instant):
   
    img_tag = soup_instant.find_all(id='icImg') 
    img = img_tag[0]['src'] if img_tag else np.nan
    return img

def retrieve_data(soup):
    
    '''retrieve data from an item'''
    total_sales = retrieve_total_sales(soup)
    sales_a_day = retrieve_sale_each_day(soup)
    price = retrieve_price(soup)
    inventory = retrieve_inventory(soup)
    shipping = retrieve_shipping(soup)
    img = retrieve_img(soup)
    company=retrieve_company_name(soup)

    return sales_a_day,total_sales,price,inventory,shipping,img,company


In [10]:
def retrieve_data_by_sel(url):
    
    options = Options()
    PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT
    options.add_argument('--proxy-server=%s' % PROXY)
    options.add_argument("--headless")
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('detach',True)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options )
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    sales_each_day = retrieve_sale_each_day(soup) 
    price = retrieve_price(soup)
    shipping = retrieve_shipping(soup)
    img = retrieve_img(soup)
    company=retrieve_company_name(soup)
    driver.close()
    
    return sales_each_day,np.nan,price,np.nan,shipping,img,company
retrieve_data_by_sel('https://www.ebay.de/itm/253431189268')

('2 verkauft in den letzten 24 Stunden', nan, nan, nan, 0, nan, nan)

In [13]:
def fetch_data(item_list,test=True,no=5):
    user_agent_list = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]
    ls_sold,ls_prc,ls_shipping,ls_totalsold,ls_available,img_list,company_list=[],[],[],[],[],[],[]
    lst=(ls_sold,ls_totalsold,ls_prc,ls_available,ls_shipping,img_list,company_list)
    max_item=no if test else len(item_list)
    for item in item_list[:max_item]:
        
        #Pick a random user agent
        #user_agent = random.choice(user_agent_list)
        #user_agent=UserAgent().random
        headers={'Request Line':'GET / HTTP/1.1',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Encoding':'gzip, deflate',
        'Accept-Language':'en-US,en;q=0.9,de-DE;q=0.8,de;q=0.7,vi;q=0.6,fr;q=0.5',
        'Connection':'keep-alive',
        #Host:myhttpheader.com
        'Upgrade-Insecure-Requests':1,
        'User-Agent':UserAgent().random}

        url = f'https://www.ebay.de/itm/{item}'
        
        time.sleep(random.choice(np.linspace(0.0,2.0,50)))
        while True:
            try:
                result = requests.get(url,proxies=renew_proxy(),timeout=3)
                print(result.request.headers)
              
                break
            except :
                print('failed')
                continue
        soup = BeautifulSoup(result.content,'html.parser')

        row = retrieve_data(soup)
        if (row[0]==0) and (row[1] is np.nan):
            row = retrieve_data_by_sel(url)

        for item,ls in zip(row,lst):
            ls.append(item)

    global num_sales_aday
    num_sales_aday=datetime.datetime.now().strftime('%Y%m%d')
    dic={f'{num_sales_aday}':ls_sold,'total_sales':ls_totalsold,'price':ls_prc,'inventory':ls_available,'shipping':ls_shipping,'item_number':item_list[:max_item],'img':img_list,'company':company_list}
    return pd.DataFrame(dic)

In [31]:
df = retrieve_item_data_by_shopname(shop_name)


109.69.67.17


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IndexError: list index out of range

In [ ]:
df.head(50)

In [15]:
data = fetch_data(df.loc[:10,'item_number'],no=10)


NameError: name 'df' is not defined

In [16]:
data.head(15)

NameError: name 'data' is not defined

In [17]:
data.head(15)


NameError: name 'data' is not defined

In [23]:

final_df= pd.concat([data.loc[:,[f'{num_sales_aday}','total_sales','inventory','img','company']],df],axis=1)


In [24]:
final_df.dropna(inplace=True)
final_df['price']=final_df['price'].str.replace(r'[a-zA-Z+]','').str.replace('  ',' ')
final_df['price']=final_df['price'].str.lstrip()
final_df['price1']=final_df['price'].str.split(expand=True)[0]
#final_df['price2']=final_df['price'].str.split(expand=True)[1]

final_df.loc[:,f'{num_sales_aday}'] = final_df.loc[:,f'{num_sales_aday}'].str.split(expand=True)[0].astype(int)
final_df.loc[:,'total_sales'] = final_df.loc[:,'total_sales'].str.split(expand=True)[0].str.replace('.','').astype(int)
final_df.loc[:,'inventory'] = final_df.loc[:,'inventory']

#final_df['shipping'] = final_df['shipping'].str.replace('KOSTENLOS','0').str.replace(r'[a-zA-Z+]','').str.replace(r',','.').astype(float)
#final_df['price1']=final_df['price1'].str.replace(',','.').astype(float)
#final_df['price2'].fillna('0',inplace=True)
#final_df['price2']= final_df['price2'].str.replace(',','.').astype(float)

/var/folders/b8/g_kq7hks5k514lxq387j61600000gn/T/ipykernel_3536/1253429841.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  final_df['price']=final_df['price'].str.replace(r'[a-zA-Z+]','').str.replace('  ',' ')
/var/folders/b8/g_kq7hks5k514lxq387j61600000gn/T/ipykernel_3536/1253429841.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  final_df.loc[:,'total_sales'] = final_df.loc[:,'total_sales'].str.split(expand=True)[0].str.replace('.','').astype(int)


In [25]:
products_variant_price_index=final_df['price2'].notna().index
final_df.loc[products_variant_price_index,'price1']= final_df.loc[products_variant_price_index,'price2']/5 + final_df.loc[products_variant_price_index,'price1']

KeyError: 'price2'

In [26]:
final_df

,20220325,total_sales,inventory,img,company,price,shipping,title,item_number,url,price1
0,4,1631,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/pdwAAOSwLdteor5...,Deuba GmbH & Co. KG,"279,95 349,95",Kostenloser Versand,"Gewächshaus 3,7m²-4,75m² Treibhaus Tomatenhaus...",253431189268,https://www.ebay.de/itm/253431189268,"279,95"
1,11,9,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/sK8AAOSwp09he6J...,Deuba GmbH & Co. KG,"61,95",Kostenloser Versand,B-Ware Holzfliesen Terrassenfliesen 3m² Garten...,275005949196,https://www.ebay.de/itm/275005949196,"61,95"
2,6,6401,\n\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/7FoAAOSwzSpg6~~...,Deuba GmbH & Co. KG,"14,95",Kostenloser Versand,12 Paar Pierre Cardin® Socken Herren Business ...,264953160503,https://www.ebay.de/itm/264953160503,"14,95"
3,9,790,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/XJIAAOSw5~BiHe6...,Deuba GmbH & Co. KG,"9,95",Kostenloser Versand,Digitalwaage Küchenwaage Briefwaage 8kg/ 1g Ha...,275081606783,https://www.ebay.de/itm/275081606783,"9,95"
4,6,263,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/Z7oAAOSwdYxiFyd...,Deuba GmbH & Co. KG,"14,95 49,95",Kostenloser Versand,Kunststoffbox Aufbewahrungsbox Stapelbox mit D...,275181346477,https://www.ebay.de/itm/275181346477,"14,95"
5,24,29,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/XqUAAOSwcZxhyvM...,Deuba GmbH & Co. KG,"189,95",Kostenloser Versand,"Geräteschuppen Gerätehaus Gartenhaus M 1,4m² M...",265473366019,https://www.ebay.de/itm/265473366019,"189,95"
6,7,4497,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/cbwAAOSwQPNiE1p...,Deuba GmbH & Co. KG,"32,95 99,95",Kostenloser Versand,Komposter Gartenkomposter Thermokomposter Schn...,253477741558,https://www.ebay.de/itm/253477741558,"32,95"
7,5,61,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/uWoAAOSwJjdiIhr...,Deuba GmbH & Co. KG,"68,95",Kostenloser Versand,Fahrrad Montageständer Reparaturständer Fahrra...,275194133227,https://www.ebay.de/itm/275194133227,"68,95"
8,5,1656,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/4O4AAOSww-1iEy3...,Deuba GmbH & Co. KG,"21,95",Kostenloser Versand,Crimpzange Aderendhülsenzange Presszange 1200 ...,254961548521,https://www.ebay.de/itm/254961548521,"21,95"
9,9,11169,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/1dkAAOSwXSFg1Ck...,Deuba GmbH & Co. KG,"34,95 37,95",Kostenloser Versand,Auflagenbox Kunststoff Truhe Box Kissenbox 310...,262755302004,https://www.ebay.de/itm/262755302004,"34,95"


In [27]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   20220325     15 non-null     int64 
 1   total_sales  15 non-null     int64 
 2   inventory    15 non-null     object
 3   img          15 non-null     object
 4   company      15 non-null     object
 5   price        15 non-null     object
 6   shipping     15 non-null     object
 7   title        15 non-null     object
 8   item_number  15 non-null     object
 9   url          15 non-null     object
 10  price1       15 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.4+ KB


In [28]:

final_df['total_revenue'] = final_df[f'{num_sales_aday}'] * final_df['price1'] + final_df['shipping']
final_df

,20220325,total_sales,inventory,img,company,price,shipping,title,item_number,url,price1,total_revenue
0,4,1631,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/pdwAAOSwLdteor5...,Deuba GmbH & Co. KG,"279,95 349,95",Kostenloser Versand,"Gewächshaus 3,7m²-4,75m² Treibhaus Tomatenhaus...",253431189268,https://www.ebay.de/itm/253431189268,"279,95","279,95279,95279,95279,95Kostenloser Versand"
1,11,9,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/sK8AAOSwp09he6J...,Deuba GmbH & Co. KG,"61,95",Kostenloser Versand,B-Ware Holzfliesen Terrassenfliesen 3m² Garten...,275005949196,https://www.ebay.de/itm/275005949196,"61,95","61,9561,9561,9561,9561,9561,9561,9561,9561,956..."
2,6,6401,\n\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/7FoAAOSwzSpg6~~...,Deuba GmbH & Co. KG,"14,95",Kostenloser Versand,12 Paar Pierre Cardin® Socken Herren Business ...,264953160503,https://www.ebay.de/itm/264953160503,"14,95","14,9514,9514,9514,9514,9514,95Kostenloser Versand"
3,9,790,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/XJIAAOSw5~BiHe6...,Deuba GmbH & Co. KG,"9,95",Kostenloser Versand,Digitalwaage Küchenwaage Briefwaage 8kg/ 1g Ha...,275081606783,https://www.ebay.de/itm/275081606783,"9,95","9,959,959,959,959,959,959,959,959,95Kostenlose..."
4,6,263,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/Z7oAAOSwdYxiFyd...,Deuba GmbH & Co. KG,"14,95 49,95",Kostenloser Versand,Kunststoffbox Aufbewahrungsbox Stapelbox mit D...,275181346477,https://www.ebay.de/itm/275181346477,"14,95","14,9514,9514,9514,9514,9514,95Kostenloser Versand"
5,24,29,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/XqUAAOSwcZxhyvM...,Deuba GmbH & Co. KG,"189,95",Kostenloser Versand,"Geräteschuppen Gerätehaus Gartenhaus M 1,4m² M...",265473366019,https://www.ebay.de/itm/265473366019,"189,95","189,95189,95189,95189,95189,95189,95189,95189,..."
6,7,4497,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/cbwAAOSwQPNiE1p...,Deuba GmbH & Co. KG,"32,95 99,95",Kostenloser Versand,Komposter Gartenkomposter Thermokomposter Schn...,253477741558,https://www.ebay.de/itm/253477741558,"32,95","32,9532,9532,9532,9532,9532,9532,95Kostenloser..."
7,5,61,\n\t\t\t\t\t\t\t\t6 verfügbar,https://i.ebayimg.com/images/g/uWoAAOSwJjdiIhr...,Deuba GmbH & Co. KG,"68,95",Kostenloser Versand,Fahrrad Montageständer Reparaturständer Fahrra...,275194133227,https://www.ebay.de/itm/275194133227,"68,95","68,9568,9568,9568,9568,95Kostenloser Versand"
8,5,1656,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/4O4AAOSww-1iEy3...,Deuba GmbH & Co. KG,"21,95",Kostenloser Versand,Crimpzange Aderendhülsenzange Presszange 1200 ...,254961548521,https://www.ebay.de/itm/254961548521,"21,95","21,9521,9521,9521,9521,95Kostenloser Versand"
9,9,11169,\n\t\t\t\t\t\t\t\tMehr als 10 verfügbar,https://i.ebayimg.com/images/g/1dkAAOSwXSFg1Ck...,Deuba GmbH & Co. KG,"34,95 37,95",Kostenloser Versand,Auflagenbox Kunststoff Truhe Box Kissenbox 310...,262755302004,https://www.ebay.de/itm/262755302004,"34,95","34,9534,9534,9534,9534,9534,9534,9534,9534,95K..."


In [29]:
for idx, row in thing_list.loc[:5,:].iterrows():
    print(row['en'],row['de'])
    

NameError: name 'thing_list' is not defined

In [ ]:
import mysql.connector as connector

config = {
    "host": "localhost",
    "user": "root",
    "password": "!mnhhtq103",
    "auth_plugin": "mysql_native_password",
    # "port": 3306,
}
conn = connector.connect(**config)
c = conn.cursor()

c.execute('USE ebay_products')

cr_table='CREATE TABLE things_list (en_word varchar(255), de_word varchar(255))'
c.execute(cr_table)

query = f'INSERT INTO things_list(en_word,de_word) VALUES '
c.execute(query)
conn.commit()
c.close()

final_df.to_csv(f'{shop_name}_data.csv')


In [ ]:
from sqlalchemy import create_engine    
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host='localhost', db='ebay_products', user='root', pw='!mnhhtq103'))

In [4]:
url='https://www.ebay.de/itm/253431189268'
        # pages
proxy=renew_proxy()
result = requests.get(url,proxies=proxy)
print(result.content)

171.25.193.235
b'<!DOCTYPE html>\n<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" lang=de>\n<head>\n<link rel="dns-prefetch" href="//i.ebayimg.com">\n<link rel="dns-prefetch" href="//thumbs.ebaystatic.com">\n<link rel="dns-prefetch" href="//vi.vipr.ebaydesc.com">\n<link rel="dns-prefetch" href="//p.ebaystatic.com">\n<link rel="dns-prefetch" href="//q.ebaystatic.com">\n<link rel="dns-prefetch" href="//pics.ebaystatic.com">\n<link rel="dns-prefetch" href="//ir.ebaystatic.com">\n<link rel="dns-prefetch" href="//srx.main.ebayrtm.com">\n<link rel="dns-prefetch" href="//rover.ebay.com">\n<link rel="dns-prefetch" href="//reco.ebay.com">\n\n<!--  fix for IE freezing.  -->\n<style type="text/css">\nbody #Body .btn, body #Body c-std {\nfilter:none;\n-ms-filter: "progid:DXImageTransform.Microsoft.gradient(startColorstr=\'#0079bc\',endColorstr=\'#00509d\')";\n}\n</style>\n\n\t<meta name="viewport"/>\n\t<meta name="layout" content="main" />\n\t<!--

Echtheit ASIN Prüfung
B00P94SAPY Beauty Blender Rechnung
B00PELSLME FEG Wimpernserum
B01I84E4FG Gesichtmaske Coldmoon 

Remissionsauftrag Papier !
Namen alten Lieferanten !
Produktechtheit 
folgende Schritte wird durchgeführt 
...
Für alle Produkte Zertifikate
Produktsicherheit 
Klarer Schritt für Schritte

Rechnungen als Nachweis für neue Distributoren


